# Optimize the Model

Achieve a target predictive accuracy higher than 75% by using "any or all" of the following:

1) Adjusting the input data to ensure that there are no variables or outliers that are causing confusion in the model, such as:
    
    Dropping more or fewer columns.
    
    Creating more bins for rare occurrences in columns.
    
    Increasing or decreasing the number of values for each bin.

2) Adding more neurons to a hidden layer.
3) Adding more hidden layers.
4) Using different activation functions for the hidden layers.
5) Adding or reducing the number of epochs to the training regimen.

Current Parameters:

In [1]:
# Cutoffs
name_cutoff = 10
application_cutoff = 200 # Original: 200, Applications types with less than this number will be added to a combined type
classification_cutoff = 1000 # Original: 1000, Classification types with less than this number will be added to a combined type

# Number of Neurons
neurons_1 = 27  # Original: 80
neurons_2 = 5  # Original: 30
neurons_3 = 3
neurons_4 = 7

# Activation Types
activation_1 = "relu" # Original: "relu"
activation_2 = "sigmoid" # Original: "relu"
activation_3 = "sigmoid"
activation_4 = "sigmoid"
activation_outer = "sigmoid" # Original: "sigmoid"

#Number of Epochs
num_epochs = 20 # Original: 50

# Preprocessing

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd
import tensorflow as tf
import keras

#  Import and read the charity_data.csv.
import pandas as pd 
# application_df = pd.read_csv("Resources/charity_data.csv")
# application_df.head()

In [5]:
df = pd.DataFrame(pd.read_excel("Data/Sample_NBA_Data_Reg_Season_2022.xlsx"))

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

The features of this model will be APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, STATUS, INCOME_AMT, SPECIAL_CONSIDERATIONS, and ASK_AMT.

The target of this model will be IS_SUCCESSFUL.

In [ ]:
application_df.dtypes

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_prep = application_df.drop(['EIN'], axis = 1)

In [ ]:
# Determine the number of unique values in each column.
application_prep.nunique()

Categorical Columns with more than 10 unique values: APPLICATION_TYPE (17), CLASSIFICATION (71)

In [ ]:
name_types = pd.Series(application_prep['NAME']).value_counts()
name_types[name_types > 10]

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced - Starter_Code cutoff was between 156 and 528
bin_name = f"Less_Than_{name_cutoff}"

# use the variable name `application_types_to_replace`
name_types_to_replace = []

for name, count in name_types.iteritems():
    if count <= name_cutoff:
        name_types_to_replace.append(name)

# Replace in dataframe
for name in name_types_to_replace:
    application_prep['NAME'] = application_prep['NAME'].replace(name, bin_name)

# Check to make sure binning was successful
application_prep['NAME'].value_counts()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_types = pd.Series(application_prep['APPLICATION_TYPE']).value_counts()
application_types

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced - Starter_Code cutoff was between 156 and 528
bin_name = f"Less_Than_{application_cutoff}"

# use the variable name `application_types_to_replace`
application_types_to_replace = []

for app, count in application_types.iteritems():
    if count < application_cutoff:
        application_types_to_replace.append(app)

# Replace in dataframe
for app in application_types_to_replace:
    application_prep['APPLICATION_TYPE'] = application_prep['APPLICATION_TYPE'].replace(app, bin_name)

# Check to make sure binning was successful
application_prep['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_types = pd.Series(application_prep['CLASSIFICATION']).value_counts()
classification_types

In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_types[classification_types > 1]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced - Starter_Code cutoff was between 1883 and 777
bin_name = f"Less_Than_{classification_cutoff}"

# use the variable name `classifications_to_replace`
classifications_to_replace = []

for classification, count in classification_types.iteritems():
    if count < classification_cutoff :
        classifications_to_replace.append(classification)

# Replace in dataframe
for classification in classifications_to_replace:
    application_prep['CLASSIFICATION'] = application_prep['CLASSIFICATION'].replace(classification,bin_name)
    
# Check to make sure binning was successful
application_prep['CLASSIFICATION'].value_counts()

In [ ]:
application_prep.head()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_prep)

In [ ]:
application_dummies.columns

### Separate Target and Features into Training and Testing Sets

In [ ]:
# Split our preprocessed data into our features and target arrays
X = application_dummies.drop(['IS_SUCCESSFUL'], axis = 1)
y = application_dummies['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
X_train_scaled.shape, X_test_scaled.shape

# Compile, Train and Evaluate the Model

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_features = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = neurons_1, activation = activation_1, input_dim = num_features))

# Second hidden layer ("If Necessary")
nn.add(tf.keras.layers.Dense(units = neurons_2, activation = activation_2))

# Third hidden layer ("If Necessary")
nn.add(tf.keras.layers.Dense(units = neurons_3, activation = activation_3))

# Third hidden layer ("If Necessary")
nn.add(tf.keras.layers.Dense(units = neurons_4, activation = activation_4))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation = activation_outer))

# Check the structure of the model
nn.summary()

In [ ]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Create a callback that saves the model's weights every 5 epochs.
saves = 5
mc = keras.callbacks.ModelCheckpoint('Optimized_Weights/weights{epoch:08d}.h5', save_weights_only=True, save_freq=saves, verbose = 0)

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=num_epochs, callbacks=[mc])

In [ ]:
# Create a DataFrame containing training history
history_df = pd.DataFrame(fit_model.history)

# Increase the index by 1 to match the number of epochs
history_df.index += 1

# Plot the loss
history_df.plot(y="loss")
history_df.plot(y="accuracy");

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Save and export your results to an HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")

In [ ]:
# Load the model
from tensorflow.keras.models import load_model
loan_model = load_model("AlphabetSoupCharity_Optimization.h5")

In [ ]:
model_loss, model_accuracy = loan_model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

## Values Before Optimization

    268/268 - 0s - loss: 0.5555 - accuracy: 0.7297
    Normal Neural Network - Loss: 0.5554825067520142, Accuracy: 0.72967928647995

## Automated Optimization

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=2), activation=activation, input_dim=num_features))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled, y_train, epochs = 20, validation_data=(X_test_scaled,y_test))

In [ ]:
X_train_scaled.shape, X_test_scaled.shape

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters()[0]
best_hyper.values

In [ ]:
def get_best_model(nn, parameters, X_train_scaled, y_train):
    clf = GridSearchCV(model, parameters, cv=4, n_jobs=-1)
    clf.fit(X_train, y_train)
    # print(clf.best_params_)
    return clf.best_estimator_ 

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

In [ ]:
# activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
# param_grid = dict(activation=activation)


# param_grid = dict(epochs=[10, 20, 30])
# grid = GridSearchCV(estimator=model, param_grid = param_grid, n_jobs=-1, cv=3)
# grid_result = grid.fit(X_train, y_train)

In [ ]:
# best_model = tuner.get_best_models()[0]
# best_model

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")